In [24]:
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import numpy as np

In [25]:
df = pd.read_csv('DS_Sample_1.csv')
df.shape

(1000, 18)

# 전처리

## Step1

In [26]:
df[['CREDIT_LIMIT', 'MINIMUM_PAYMENTS']] = df[['CREDIT_LIMIT', 'MINIMUM_PAYMENTS']].apply(lambda x: x.fillna(x.mean()))

In [27]:
df['MINIMUM_PAYMENTS'].isna().sum()

0

In [28]:
card1 = df.copy()

# Q1

In [29]:
card1.groupby('TENURE')['BALANCE', 'CREDIT_LIMIT'].corr()[::2]['CREDIT_LIMIT'].max().round(2)

0.95

# Q2

## Step1

In [30]:
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [31]:
df_2 = card1.iloc[:, 1:]
df_2.head()

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0.0,2.0,1000.0,201.802084,139.509787,0.000000,12.0
1,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4.0,0.0,7000.0,4103.032597,1072.340217,0.222222,12.0
2,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0.0,12.0,7500.0,622.066742,627.284787,0.000000,12.0
3,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1.0,1.0,7500.0,0.000000,1297.116322,0.000000,12.0
4,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0.0,1.0,1200.0,678.334763,244.791237,0.000000,12.0


In [32]:
cols = df_2.columns
cols

Index(['BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES', 'ONEOFF_PURCHASES',
       'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE', 'PURCHASES_FREQUENCY',
       'ONEOFF_PURCHASES_FREQUENCY', 'PURCHASES_INSTALLMENTS_FREQUENCY',
       'CASH_ADVANCE_FREQUENCY', 'CASH_ADVANCE_TRX', 'PURCHASES_TRX',
       'CREDIT_LIMIT', 'PAYMENTS', 'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT',
       'TENURE'],
      dtype='object')

In [33]:
df_scaled = pd.DataFrame(StandardScaler().fit_transform(df_2), columns=cols)

## Step2

In [34]:
for n in range(2, 6):
    model = KMeans(n_clusters=n, random_state=1234).fit(df_scaled)
    score = silhouette_score(df_scaled, model.labels_)
    
    print(n, score)

2 0.3075281530456079
3 0.19636128772937608
4 0.20715098494639902
5 0.19274056144483248


In [35]:
k = 2

## Step3

In [36]:
model = KMeans(n_clusters=2, random_state=1234).fit(df_scaled)

In [37]:
df_2['cluster'] = model.labels_

In [38]:
df_2.groupby('cluster')['ONEOFF_PURCHASES'].mean().max().round(2)

3946.19

# Q3

## Step1

In [39]:
card1.head()

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,10001.0,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0.0,2.0,1000.0,201.802084,139.509787,0.000000,12.0
1,10002.0,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4.0,0.0,7000.0,4103.032597,1072.340217,0.222222,12.0
2,10003.0,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0.0,12.0,7500.0,622.066742,627.284787,0.000000,12.0
3,10004.0,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1.0,1.0,7500.0,0.000000,1297.116322,0.000000,12.0
4,10005.0,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0.0,1.0,1200.0,678.334763,244.791237,0.000000,12.0


In [40]:
train = card1[card1['CUST_ID'] % 4 != 0]
test = card1[card1['CUST_ID'] % 4 == 0]

## Step2

In [41]:
from sklearn.tree import DecisionTreeRegressor

In [42]:
x_col = card1.set_index('ONEOFF_PURCHASES').reset_index().columns[2:]
len(x_col)

16

In [43]:
model = DecisionTreeRegressor(random_state=1234).fit(X=train[x_col], y=train['ONEOFF_PURCHASES'])

## Step3

In [44]:
from sklearn.metrics import mean_squared_error

In [45]:
pred = model.predict(test[x_col])

In [47]:
round((mean_squared_error(y_true=test['ONEOFF_PURCHASES'], y_pred=pred))**0.5, 1)

2383.8